In [1]:
import cv2
from keras.models import load_model
import numpy as np
from collections import deque

Using TensorFlow backend.


In [2]:
model1 = load_model('devanagari.h5')
print(model1)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [3]:

letter_count = {0: 'CHECK', 1: '01_ka', 2: '02_kha', 3: '03_ga', 4: '04_gha', 5: '05_kna', 6: 'character_06_cha',
                    7: '07_chha', 8: '08_ja', 9: '09_jha', 10: '10_yna',
                    11: '11_taamatar',
                    12: '12_thaa', 13: '13_daa', 14: '14_dhaa', 15: '15_adna', 16: '16_tabala', 17: '17_tha',
                    18: '18_da',
                    19: '19_dha', 20: '20_na', 21: '21_pa', 22: '22_pha',
                    23: '23_ba',
                    24: '24_bha', 25: '25_ma', 26: '26_yaw', 27: '27_ra', 28: '28_la', 29: '29_waw', 30: '30_motosaw',
                    31: '31_petchiryakha',32: '32_patalosaw', 33: '33_ha',
                    34: '34_chhya', 35: '35_tra', 36: '36_gya', 37: 'CHECK'}


In [ ]:
cap = cv2.VideoCapture(0)
Lower_green = np.array([110, 50, 50])
Upper_green = np.array([130, 255, 255])
pred_class=0
pts = deque(maxlen=512)
blackboard = np.zeros((480, 640, 3), dtype=np.uint8)
digit = np.zeros((200, 200, 3), dtype=np.uint8)
while (cap.isOpened()):
    ret, img = cap.read()
    img = cv2.flip(img, 1)
    if ret:
        imgHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(imgHSV, Lower_green, Upper_green)
        blur = cv2.medianBlur(mask, 15)
        blur = cv2.GaussianBlur(blur, (5, 5), 0)
        thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]
        center = None
        if len(cnts) >= 1:
            contour = max(cnts, key=cv2.contourArea)
            if cv2.contourArea(contour) > 250:
                ((x, y), radius) = cv2.minEnclosingCircle(contour)
                cv2.circle(img, (int(x), int(y)), int(radius), (0, 255, 255), 2)
                cv2.circle(img, center, 5, (0, 0, 255), -1)
                M = cv2.moments(contour)
                center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))
                pts.appendleft(center)
                for i in range(1, len(pts)):
                    if pts[i - 1] is None or pts[i] is None:
                        continue
                    cv2.line(blackboard, pts[i - 1], pts[i], (255, 255, 255), 10)
                    cv2.line(img, pts[i - 1], pts[i], (0, 0, 255), 5)
        elif len(cnts) == 0:
            if len(pts) != []:
                blackboard_gray = cv2.cvtColor(blackboard, cv2.COLOR_BGR2GRAY)
                blur1 = cv2.medianBlur(blackboard_gray, 15)
                blur1 = cv2.GaussianBlur(blur1, (5, 5), 0)
                thresh1 = cv2.threshold(blur1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
                blackboard_cnts = cv2.findContours(thresh1.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]
                if len(blackboard_cnts) >= 1:
                    cnt = max(blackboard_cnts, key=cv2.contourArea)
                    print(cv2.contourArea(cnt))
                    if cv2.contourArea(cnt) > 2000:
                        x, y, w, h = cv2.boundingRect(cnt)
                        digit = blackboard_gray[y:y + h, x:x + w]
                        # newImage = process_letter(digit)
                        pred_probab, pred_class = keras_predict(model1, digit)
                        print(pred_class, pred_probab)

            pts = deque(maxlen=512)
            blackboard = np.zeros((480, 640, 3), dtype=np.uint8)
        cv2.putText(img, "Conv Network :  " + str(letter_count[pred_class]), (10, 470),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.imshow("Frame", img)
        cv2.imshow("Contours", thresh)
        k = cv2.waitKey(10)
        if k == 27:
            break
    else:
        break
        
cap.release() 
cv2.destroyAllWindows() 

8654.5
processed: (1, 32, 32, 1)
26 1.0
17085.5
processed: (1, 32, 32, 1)
25 1.0
23369.0
processed: (1, 32, 32, 1)
5 1.0
2866.5
processed: (1, 32, 32, 1)
18 1.0
22263.0
processed: (1, 32, 32, 1)
7 1.0
664.5
19942.5
processed: (1, 32, 32, 1)
1 1.0
13949.0
processed: (1, 32, 32, 1)
20 1.0
8889.0
processed: (1, 32, 32, 1)
6 1.0
5378.5
processed: (1, 32, 32, 1)
27 1.0
4855.0
processed: (1, 32, 32, 1)
16 1.0
2996.0
processed: (1, 32, 32, 1)
18 1.0
9671.5
processed: (1, 32, 32, 1)
36 1.0
6332.5
processed: (1, 32, 32, 1)
6 1.0
6114.0
processed: (1, 32, 32, 1)
5 1.0
8119.5
processed: (1, 32, 32, 1)
27 1.0
18264.5
processed: (1, 32, 32, 1)
36 1.0
3475.0
processed: (1, 32, 32, 1)
9 1.0
12538.5
processed: (1, 32, 32, 1)
32 1.0
8012.0
processed: (1, 32, 32, 1)
7 1.0
3931.5
processed: (1, 32, 32, 1)
7 1.0
7171.0
processed: (1, 32, 32, 1)
1 1.0
2240.0
processed: (1, 32, 32, 1)
34 1.0
2494.0
processed: (1, 32, 32, 1)
10 1.0
16393.5
processed: (1, 32, 32, 1)
36 1.0
3301.0
processed: (1, 32, 32, 1)
31 

In [4]:
def keras_predict(model, image):
    processed = keras_process_image(image)
    print("processed: " + str(processed.shape))
    pred_probab = model.predict(processed)[0]
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class


def keras_process_image(img):
    image_x = 32
    image_y = 32
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img